# Atlas API (v2)


* [API overview](https://atlas.ripe.net/docs/api/v2/manual/overview/)

* [API reference](https://atlas.ripe.net/docs/api/v2/reference/)
* [API keys](https://atlas.ripe.net/docs/keys/)
> You can use the API as an anonymous user to request only publicly available information
  To create measurements and to see information about your own probes, you will have to 
  identify yourself to RIPE Atlas. One of the ways is the API keys. You can get them yourself.
  First you have to [create an Atlas account](https://access.ripe.net/registration) and then [generate your keys](https://atlas.ripe.net/keys/)
   
 > an API key can be provided in the query string like `key=427e1e6c-d3b5-428e-a33f-81ed1d18e402`
 
 > **you will need to specify it below to make all examples working**
   
* [Probe API](#probe_api)
 - [Request all probes metadata](#probe_api_all)
 - [Get a probe by ID](#probe_api_byid)
 - [Request probes within 10km radius of Amsterdam](#probe_api_radius)
 - [Get online working native IPv6 probes](#probe_api_tags)
 
* [Measurements API](#msm_api)
 - [Get my measurements definitions applying filters and options](#msm_get_my)
 - [Create a measurement](#msm_create_oneoff)
 - [Getting measurement results](#msm_get_results)
* [Python libraries for RIPE Atlas API](#python_libs)
 - [**Cousteau** - RIPE Atlas API Python wrapper](http://ripe-atlas-cousteau.readthedocs.io/en/latest/)
> This is an official package for accessing RIPE Atlas API. Maintained by RIPE Atlas developers.
[**Cousteau** GitHub](https://github.com/RIPE-NCC/ripe-atlas-cousteau)

 - [**Sagan** - Parsing library for RIPE Atlas measurement results](https://ripe-atlas-sagan.readthedocs.io/en/latest/)
 > This is an official package for parsing RIPE Atlas measurement results. Maintained by RIPE Atlas developers.
[**Sagan** GitHub](https://github.com/RIPE-NCC/ripe.atlas.sagan)

 - [Create a measurement using **Cousteau**](#msm_create_oneoff_cousteau)
 - [Streaming results using **Cousteau**](#msm_stream_cousteau)
 - [Parsing results using **Sagan**](#msm_parsing_sagan)

In [ ]:
# import some useful stuff
from IPython.display import display
import requests
import json

# your API key
key="<your api key>"

<a id='probe_api'></a>
## Probe API

<a id='probe_api_all'></a>
### Request all probes metadata

In [ ]:
r = requests.get('https://atlas.ripe.net/api/v2/probes/')
probe_data = r.json()

print "Total probes: {}".format(probe_data['count'])
# get first probe from the list and display it
print "Metadata of probe #{}:".format(probe_data['results'][1]["id"])
display(probe_data['results'][1])


<a id='probe_api_byid'></a>
### Get a probe by ID

In [ ]:
r = requests.get('https://atlas.ripe.net/api/v2/probes/22344')
display(r.json())

<a id='probe_api_radius'></a>
### Request probes within 10km radius of Amsterdam

> **Hint!** if you click on the URL you will be able to browse the API

In [ ]:
lat, lon = (52.36, 4.9) # latitude and longitude in degrees
radius = 10             # radius in km

r = requests.get('https://atlas.ripe.net:443/api/v2/probes/?radius={},{}:{}'.format(lat, lon, radius))
print "The URL is {}".format(r.url)
probe_data = r.json()
print "Total probes inside 10km radius near Amsterdam: {}".format(probe_data['count'])


<a id='probe_api_tags'></a>
### Get online working native IPv6 probes

In [ ]:
# specify tags
tags = ["system-ipv6-works", "native-ipv6"]

# returns online working native IPv6 probes
r = requests.get('https://atlas.ripe.net/api/v2/probes/',
    params=dict(
        status=1,
        tags=",".join(tags)
    )
)
print "The URL is {}".format(r.url)
probe_data = r.json()
print "Number of online working native IPv6 probes: {}".format(probe_data['count'])


<a id='msm_api'></a>
# Measurements API

<a id='msm_get_my'></a>
## Get my measurements definitions applying filters and options

In [ ]:
r = requests.get('https://atlas.ripe.net/api/v2/measurements/my/',
    params = dict(   
        page_size=3,              # Get first 3 definitions
        page=1,                   #   from page1
        af=6,                     # filter for IPv6 measurements
        sort="-id",               # apply reverse sorting by measurement id
        fields='type,target,af',  # show only measurement type and target fields
        optional_fields='participation_requests', # include optional fields
        key=key
    )
)
display(r.json())

<a id='msm_create_oneoff'></a>
## Create a measurement

In [ ]:
#POST /api/v2/measurements/

msm_def = dict(
    is_oneoff=True,
    definitions=[
        dict(
            description='ping to xs4all',
            af=4,
            type='ping',
            target='ping.xs4all.nl'
        )
    ],
    probes=[
        dict(
            action='add',
            type='area',
            value='West',
            requested=5
            
        ),
        dict(
            action='add',
            type='country',
            value='NL',
            requested=3 
        )
    ]
)

r = requests.post(
    'https://atlas.ripe.net/api/v2/measurements/',
    params = {'key': key},
    data=json.dumps(msm_def),
    headers = {"Content-Type": "application/json"}
)

# expected result:
# {u'measurements': [4599188]}

print r.text
measurements = r.json()['measurements']

In [ ]:
print "Getting new measurement metadata"

r = requests.get(
    'https://atlas.ripe.net/api/v2/measurements/{}/'.format(measurements[0]),
    params = dict(   
        optional_fields='participation_requests', # include optional fields
        key=key
    )
)
display(r.json())

<a id='msm_get_results'></a>
## Getting measurement results

In [ ]:

r = requests.get(
    'https://atlas.ripe.net/api/v2/measurements/{}/results'.format(measurements[0])
)

display(r.json())

<a id='python_libs'></a>
# Python libraries for RIPE ATLAS API


 <a id='msm_stream_cousteau'></a>
## Create a measurement using **Cousteau**
> create the same measurement as in the example above

In [ ]:
from ripe.atlas.cousteau import Ping, AtlasSource, AtlasCreateRequest

# measurement definition
ping = Ping(af=4, target="www.google.gr", description="testing new wrapper")
# measurement sources
source_area = AtlasSource(type="area", value="West", requested=5)
source_country = AtlasSource(type="country", value="NL", requested=3)

atlas_request = AtlasCreateRequest(
    key=key,
    measurements=[ping],
    sources=[source_area, source_country],
    is_oneoff=True
)

(is_success, response) = atlas_request.create()
display(is_success, response)

 <a id='msm_create_oneoff_cousteau'></a>
## Streaming results using **Cousteau**

In [ ]:
from ripe.atlas.cousteau import AtlasStream

def on_result_response(*args):
    """
    Function that will be called every time we receive a new result.
    Args is a tuple, so you should use args[0] to access the real message.
    """
    display(args[0])

atlas_stream = AtlasStream()
atlas_stream.connect()
# Measurement results
channel = "result"
# Bind function we want to run with every result message received
atlas_stream.bind_channel(channel, on_result_response)
# Subscribe to new stream for 1001 measurement results
stream_parameters = {"msm": 1001}
atlas_stream.start_stream(stream_type="result", **stream_parameters)

# Probe's connection status results
channel = "probe"
atlas_stream.bind_channel(channel, on_result_response)
stream_parameters = {"enrichProbes": True}
atlas_stream.start_stream(stream_type="probestatus", **stream_parameters)

# Execution blocks here for 5 seconds to allow receiving results and pass it
# to callback: on_result_response()
# if you want to block forever use timeout() without arguments
atlas_stream.timeout(seconds=5)
# Shut down everything
atlas_stream.disconnect()

 <a id='msm_parsing_sagan'></a>
## Parsing results using **Sagan**

In [ ]:
from datetime import datetime
from ripe.atlas.cousteau import AtlasResultsRequest
from ripe.atlas.sagan import Result

# specify what should be requested
result_request = AtlasResultsRequest(
    msm_id=1001,         # measurement of interest
    probe_ids=[1,2,3,4],  # probes of interest
    # time interval
    start=datetime(2016, 5, 19, 0, 0, 0),
    stop=datetime(2016, 5, 19, 0, 10, 0)
)

# fetch it
is_success, results = result_request.create()

# process received data
if is_success:
    # loop over received results
    print "{:^20} {:^12}".format("Origin", "Average RTT")
    for result in results:
        # and parse it
        r = Result.get(result)
        print "{:<20} {:12.3f}".format(r.origin, r.rtt_average)